In [43]:
import pandas as pd
import numpy as np

from ast import literal_eval

import statsmodels.formula.api as smf

import setup_modules
from lib.fomc import get_lname

In [44]:
fomc = pd.read_csv('../working-csvs/fomc.csv', index_col=0)

In [46]:
mfdff = pd.read_pickle('../working-csvs/mfdff_selected.pkl')

In [47]:
mfdff

,date,section,lname,svect,voter,sent,region,female,chair,exp,...,svect_tprob_38,svect_tprob_39,svect_tprob_40,svect_tprob_41,svect_tprob_42,svect_tprob_43,svect_tprob_44,final_norm_svect,final_diff_exp_norm,final_diff_prior
19,1994-02-04,2,greenspan,"[-0.29740134824317155, -0.32435034602443663, 0...",1.0,21,governor,0.0,Greenspan,6.490411,...,-1.000000,0.000000,0.0,-1.549192,0.0,-0.500000,-0.492063,"[-0.42614366417052835, 0.0, -0.288675134594812...","[-0.3573212656919145, 0.0, -0.4690970937165708...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
25,1994-02-04,2,mcdonough,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,1,New York,0.0,Greenspan,0.547945,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.000000,"[-0.42614366417052835, 0.0, -0.288675134594812...","[-0.3573212656919145, 0.0, -0.4690970937165708...","[0.0, 0.0, 0.0, 2.394498089936103, 0.0, 2.7716..."
26,1994-02-04,2,melzer,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,3,St. Louis,0.0,Greenspan,8.684932,...,0.650722,0.000000,0.0,0.000000,0.0,0.000000,-0.377256,"[-0.42614366417052835, 0.0, -0.288675134594812...","[-0.3573212656919145, 0.0, -0.4690970937165708...","[0.0, 0.0, 0.0, 1.1972490449680515, 0.0, 1.385..."
20,1994-02-04,2,jordan,"[0.0, 0.0, 0.3787425226506579, 0.0, 0.0, 0.0, ...",1.0,1,Cleveland,0.0,Greenspan,1.906849,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.621257,"[2.255153844511024, 0.0, -0.28867513459481275,...","[2.3239762429896382, 0.0, -0.4690970937165708,...","[2.6812975086815527, 0.0, 0.0, 0.7981660299787..."
16,1994-02-04,2,boehne,"[0.0, 0.390893463834136, 0.0, 0.0, 0.0, 0.0, 0...",0.0,2,Philadelphia,0.0,Greenspan,13.016438,...,-1.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,"[-0.42614366417052835, 0.0, 3.464101615137753,...","[-0.3573212656919145, 0.0, 3.283679656015995, ...","[-0.6703243771703882, 0.0, 3.7527767497325657,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5655,2018-12-19,2,george,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,3,Kansas City,1.0,Powell,7.221918,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,"[-0.1524013671639829, -0.06732518572553808, 0....","[-0.06933086735034355, -0.3202906983772544, 0....","[0.056775600484502975, 1.3877787807814457e-17,..."
5661,2018-12-19,2,quarles,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,4,governor,0.0,Powell,1.254795,...,0.000000,0.000000,0.0,1.000000,0.0,-0.457627,0.000000,"[-0.1524013671639829, -0.06732518572553808, 0....","[-0.06933086735034355, -0.3202906983772544, 0....","[0.05161418225863906, 1.3877787807814457e-17, ..."
5658,2018-12-19,2,kashkari,"[-0.3987878854106168, 0.0, 0.0, 0.0, -0.520186...",0.0,6,Minneapolis,0.0,Powell,2.967123,...,-0.602902,0.000000,0.0,0.000000,0.0,0.465753,0.000000,"[-0.1524013671639829, -2.2603543836150104, 0.5...","[-0.06933086735034355, -2.5133198962667267, 0....","[0.0473130004037525, -2.1930291978894725, 0.70..."
5663,2018-12-19,2,williams,"[0.0, 0.0, 0.42477875017013805, 0.0, 0.7597292...",1.0,24,New York,0.0,Powell,7.805479,...,0.000000,0.562115,0.0,0.000000,0.0,0.818406,1.170169,"[2.7013751451407892, 3.1355817980470055, 0.742...","[2.7844456449544284, 2.8826162853952892, 0.953...","[2.897450051139005, 3.3716015374563493, 0.8165..."


In [48]:
mfdff.columns

Index(['date', 'section', 'lname', 'svect', 'voter', 'sent', 'region',
       'female', 'chair', 'exp',
       ...
       'svect_tprob_38', 'svect_tprob_39', 'svect_tprob_40', 'svect_tprob_41',
       'svect_tprob_42', 'svect_tprob_43', 'svect_tprob_44',
       'final_norm_svect', 'final_diff_exp_norm', 'final_diff_prior'],
      dtype='object', length=127)

In [49]:
mfdff['dist'] = mfdff['final_norm_svect'].map(lambda x : np.sum(np.abs(x)) / len(x))
mfdff['dist_exp'] = mfdff['final_diff_exp_norm'].map(lambda x: np.sum(np.abs(x)) / len(x))
mfdff['dist_prior'] = mfdff['norm_svect_diff_avg_prior'].map(lambda x: np.sum(np.abs(x)) / len(x))

/tmp/ipykernel_17563/3301536847.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mfdff['dist'] = mfdff['final_norm_svect'].map(lambda x : np.sum(np.abs(x)) / len(x))
/tmp/ipykernel_17563/3301536847.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mfdff['dist_exp'] = mfdff['final_diff_exp_norm'].map(lambda x: np.sum(np.abs(x)) / len(x))
/tmp/ipykernel_17563/3301536847.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

In [50]:
mfdff['dist_exp'].describe()


count    2337.000000
mean        0.536245
std         0.228099
min         0.000000
25%         0.370268
50%         0.489690
75%         0.653463
max         1.794051
Name: dist_exp, dtype: float64

In [51]:
mfdff['date'] = pd.to_datetime(mfdff['date'])
mfdff['merge_date'] = mfdff['date'].dt.strftime('%Y-%m')
mfdff.drop(columns='date', inplace=True)

/tmp/ipykernel_17563/2449098506.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mfdff['merge_date'] = mfdff['date'].dt.strftime('%Y-%m')


In [52]:
gb = pd.read_csv('../working-csvs/greenbook.csv', index_col=0)

In [53]:
gb

,merge_date,UNEMPF0,gPPCEF0
0,1967-03,3.8,NaN
1,1967-04,4.0,NaN
2,1967-05,3.9,NaN
3,1967-06,3.9,NaN
4,1967-07,3.9,NaN
...,...,...,...
468,2018-06,3.8,2.0
469,2018-07,3.8,1.4
470,2018-09,3.8,1.5
471,2018-10,3.6,1.9


In [54]:
df = mfdff.merge(gb, how='left', on='merge_date')

In [55]:
panel = df[(df['section'] == 2) & (df['use'] == True)].copy()

In [56]:
panel

,section,lname,svect,voter,sent,region,female,chair,exp,Econ_PhD,...,svect_tprob_44,final_norm_svect,final_diff_exp_norm,final_diff_prior,dist,dist_exp,dist_prior,merge_date,UNEMPF0,gPPCEF0
0,2,greenspan,"[-0.29740134824317155, -0.32435034602443663, 0...",1.0,21,governor,0.0,Greenspan,6.490411,1.0,...,-0.492063,"[-0.42614366417052835, 0.0, -0.288675134594812...","[-0.3573212656919145, 0.0, -0.4690970937165708...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1.122381,1.081271,NaN,1994-02,NaN,NaN
2,2,melzer,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,3,St. Louis,0.0,Greenspan,8.684932,0.0,...,-0.377256,"[-0.42614366417052835, 0.0, -0.288675134594812...","[-0.3573212656919145, 0.0, -0.4690970937165708...","[0.0, 0.0, 0.0, 1.1972490449680515, 0.0, 1.385...",0.439177,0.485459,0.783896,1994-02,NaN,NaN
7,2,lindsey,"[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1.0,4,governor,0.0,Greenspan,2.194521,1.0,...,1.000000,"[-0.42614366417052835, 0.0, -0.288675134594812...","[-0.3573212656919145, 0.0, -0.4690970937165708...","[-0.38304250124022177, 0.0, -0.536110964247509...",0.343624,0.454866,0.420673,1994-02,NaN,NaN
14,2,greenspan,"[0.0, 0.0, 0.37862138877111157, 0.0, 0.0, 0.0,...",1.0,20,governor,0.0,Greenspan,6.616438,1.0,...,-0.378621,"[0.5834386687222536, 0.0, -3.1622776601683795,...","[0.5809131909484005, 0.0, -2.981576079587329, ...","[1.2245094375912153, 0.0, -3.4785054261852175,...",1.267136,1.175664,0.926576,1994-03,6.5,NaN
16,2,parry,"[-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.289495...",1.0,5,San Francisco,0.0,Greenspan,8.131507,1.0,...,0.633558,"[-0.6410707688689617, 0.0, 0.3162277660168379,...","[-0.6435962466428148, 0.0, 0.4969293465978882,...","[-0.4081698125304051, 0.0, 1.1595018087284057,...",0.456515,0.502220,0.641827,1994-03,6.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,2,bowman,"[0.30914231879903664, 0.0, 0.0, 0.0, 0.0, 0.0,...",1.0,5,governor,1.0,Powell,0.063014,0.0,...,0.000000,"[-0.1524013671639829, -0.06732518572553808, 0....","[-0.06933086735034355, -0.3202906983772544, 1....","[0.06308400053833663, 0.0, 1.206769732446244, ...",0.630941,0.748798,0.439426,2018-12,3.7,1.4
2335,2,george,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,3,Kansas City,1.0,Powell,7.221918,0.0,...,0.000000,"[-0.1524013671639829, -0.06732518572553808, 0....","[-0.06933086735034355, -0.3202906983772544, 0....","[0.056775600484502975, 1.3877787807814457e-17,...",0.604250,0.680392,0.393456,2018-12,3.7,1.4
2336,2,quarles,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,4,governor,0.0,Powell,1.254795,0.0,...,0.000000,"[-0.1524013671639829, -0.06732518572553808, 0....","[-0.06933086735034355, -0.3202906983772544, 0....","[0.05161418225863906, 1.3877787807814457e-17, ...",0.382703,0.489762,0.494997,2018-12,3.7,1.4
2337,2,kashkari,"[-0.3987878854106168, 0.0, 0.0, 0.0, -0.520186...",0.0,6,Minneapolis,0.0,Powell,2.967123,0.0,...,0.000000,"[-0.1524013671639829, -2.2603543836150104, 0.5...","[-0.06933086735034355, -2.5133198962667267, 0....","[0.0473130004037525, -2.1930291978894725, 0.70...",0.600951,0.714648,0.587486,2018-12,3.7,1.4


In [57]:
panel['gPPCEF0'].notna().value_counts()

gPPCEF0
True     1132
False     336
Name: count, dtype: int64

In [58]:
panel = panel[(panel['gPPCEF0'].notna()) & (panel['UNEMPF0'].notna())].copy()

In [59]:
panel.rename(columns={'gPPCEF0': 'pceinf', 'UNEMPF0': 'unrate'}, inplace=True)

In [60]:
panel['dist'].describe()

count    1132.000000
mean        0.604312
std         0.227002
min         0.000000
25%         0.442547
50%         0.569346
75%         0.727080
max         1.894893
Name: dist, dtype: float64

In [61]:
cov_name_map = {'exp': 'Experience', 'unrate': 'Unemployment',
                'I(exp ** 2)': 'Experience$^2$', 'pceinf': 'PCE inflation', 'C(female)[T.1.0]': 'Female', 'C(voter)[T.1]': 'Voter', 'I(exp ** 2):pceinf': 'Experience$^2$ $\\times$ PCE inflation',
                'I(exp ** 2):unrate': 'Experience$^2$ $\\times$ Unemployment', 'exp:pceinf': 'Experience $\\times$ PCE Inflation', 'exp:unrate': 'Experience $\\times$ Unemployment',
                "C(chair, Treatment(reference='Greenspan'))[T.Yellen]": "Yellen", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]": 'Bernanke', "C(chair, Treatment(reference='Greenspan'))[T.Powell]": "Powell",
                "C(Econ_PhD)[T.1.0]": 'Econ PhD', "C(voter)[T.1.0]": 'Voter'
                }

In [62]:
model1 = smf.ols("dist ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region) + C(Econ_PhD)", panel).fit(cov_type='cluster', cov_kwds={'groups': panel['merge_date']})
model2 = smf.ols("dist ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region) + C(Econ_PhD)", panel).fit(cov_type='cluster', cov_kwds={'groups': panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the $\ell_1$ distance of the stance vectors of individual committee participants from the committee average in the monetary policy discussion section divided by the number of components of the stance vector. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Committee Average Stance')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1.0]", "C(Econ_PhD)[T.1.0]","C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('base.tex', 'w') as f:
    f.write(sg.render_latex())
sg

In [63]:
panel['expd']

474      True
481      True
483     False
486      True
490      True
        ...  
2334    False
2335     True
2336    False
2337    False
2338     True
Name: expd, Length: 1132, dtype: bool

In [64]:
inexp_panel = panel[panel['expd'] == False]

In [65]:
inexp_panel['unrate'].describe()

count    384.000000
mean       6.413281
std        1.927689
min        3.700000
25%        4.800000
50%        5.700000
75%        8.200000
max       10.100000
Name: unrate, dtype: float64

In [81]:
inexp_panel = inexp_panel[inexp_panel['dist_exp'].notna()]

In [83]:
model1 = smf.ols("dist_exp ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region) + C(Econ_PhD)", inexp_panel).fit(cov_type='cluster', cov_kwds={'groups': inexp_panel['merge_date']})
model2 = smf.ols("dist_exp ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region) + C(Econ_PhD)", inexp_panel).fit(cov_type='cluster', cov_kwds={'groups': inexp_panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the $\ell_1$ distance of the stance vectors of individual committee participants from the average of members of above first-tertile experience in the monetary policy discussion section divided by the number of components of the stance vector. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Committee Average Stance of Experienced Members')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1.0]", "C(Econ_PhD)[T.1.0]", "C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('inexp.tex', 'w') as f:
    f.write(sg.render_latex())
sg

In [84]:
prior_panel = panel[panel['dist_prior'].notna()]

In [85]:
prior_panel['exp'].describe()

count    1028.000000
mean        5.424244
std         4.012816
min         0.002740
25%         2.268493
50%         4.742466
75%         7.739726
max        24.276712
Name: exp, dtype: float64

In [87]:
model1 = smf.ols("dist_prior ~ (exp + I(exp**2)) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + unrate + pceinf + C(region) + C(Econ_PhD)", prior_panel).fit(cov_type='cluster', cov_kwds={'groups': prior_panel['merge_date']})
model2 = smf.ols("dist_prior ~ (exp + I(exp**2))*(unrate + pceinf) + C(chair, Treatment(reference='Greenspan')) + C(voter) + C(female) + C(region) + C(Econ_PhD)", prior_panel).fit(cov_type='cluster', cov_kwds={'groups': prior_panel['merge_date']})

from stargazer.stargazer import Stargazer


sg = Stargazer([model1, model2])

sg.add_custom_notes(["This regression table reports results from regressions of the $\ell_1$ distance of the stance vectors of individual committee participants from the average stance of the members who spoke before them in the monetary policy discussion section divided by the number of components of the stance vector. Key explanatory variables account for experience serving on the FOMC and economic conditions related to the Federal Reserve's dual mandate as measured by unemployment and PCE inflation calculated as the percentage change from 1 year ago. Additional controls include whether the given member was a voting member at that meeting, gender, and indicator variables for each FOMC chair."])
sg.rename_covariates(cov_name_map)
sg.dependent_variable_name('Distance from Average of Prior Speakers')
sg.covariate_order(["exp",  "unrate", "pceinf", 'exp:unrate', 'exp:pceinf', "I(exp ** 2)",'I(exp ** 2):pceinf', 'I(exp ** 2):unrate', "C(female)[T.1.0]", "C(voter)[T.1.0]", "C(Econ_PhD)[T.1.0]","C(chair, Treatment(reference='Greenspan'))[T.Bernanke]", "C(chair, Treatment(reference='Greenspan'))[T.Yellen]", "C(chair, Treatment(reference='Greenspan'))[T.Powell]", 'Intercept'])
sg.title('Unordered Results')
sg.show_degrees_of_freedom(False)
sg.add_line('Region FEs', ['Yes', 'Yes'])
with open('prior.tex', 'w') as f:
    f.write(sg.render_latex())
sg